In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#list of plots in GA LTER dataset (not including broad creek)

ee.Initialize()

ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
])
##for GA, lats and longs may differ slightly between years (compare average lat/long to recorded lat/longs in each yr)
##These are average lat/longs of the site (see GIS file); as such, need zonal spectral data (not point data)
##Lat/longs calculated in GA Biomass dataset Jupyter script

In [3]:
##sentinel 2 surface reflectance images - GA LTER

gaCollectionS2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filter(ee.Filter.calendarRange(2015, 2022,'year')) \
    .filter(ee.Filter.calendarRange(9, 11,'month')) \
    .filterBounds(ga_pts) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B8', 'B4', 'B3'], 
             'gamma': 1.5}

print(gaCollectionS2.size().getInfo())
gaimage0 = gaCollectionS2.first()

55


In [4]:
gaCollectionS2.aggregate_array('system:id').getInfo()


['COPERNICUS/S2_SR/20211109T160459_20211109T161040_T17RMQ',
 'COPERNICUS/S2_SR/20211124T160611_20211124T161250_T17RMQ',
 'COPERNICUS/S2_SR/20191120T160549_20191120T160546_T17RMQ',
 'COPERNICUS/S2_SR/20211129T160619_20211129T161236_T17RMQ',
 'COPERNICUS/S2_SR/20191110T160459_20191110T160501_T17RMQ',
 'COPERNICUS/S2_SR/20201030T160411_20201030T160857_T17RMQ',
 'COPERNICUS/S2_SR/20201104T160439_20201104T160438_T17RMQ',
 'COPERNICUS/S2_SR/20211114T160531_20211114T161129_T17RMQ',
 'COPERNICUS/S2_SR/20201124T160609_20201124T161304_T17RMQ',
 'COPERNICUS/S2_SR/20200930T160101_20200930T161352_T17RMQ',
 'COPERNICUS/S2_SR/20201114T160529_20201114T161142_T17RMQ',
 'COPERNICUS/S2_SR/20210925T160021_20210925T161233_T17RMQ',
 'COPERNICUS/S2_SR/20191130T160619_20191130T160655_T17RMQ',
 'COPERNICUS/S2_SR/20190916T155911_20190916T160741_T17RMQ',
 'COPERNICUS/S2_SR/20191125T160611_20191125T161352_T17RMQ',
 'COPERNICUS/S2_SR/20211015T160241_20211015T160450_T17RMQ',
 'COPERNICUS/S2_SR/20190906T155901_20190

In [5]:
##Sentinel imagery
s2ga19 = ee.Image('COPERNICUS/S2_SR/20190926T160021_20190926T160854_T17RMQ') ##Tide = 0.17m; SEP; also close to landsat date

props = geemap.image_props(s2ga19)
props.getInfo()

{'AOT_RETRIEVAL_ACCURACY': 0,
 'CLOUDY_PIXEL_PERCENTAGE': 13.894045,
 'CLOUD_COVERAGE_ASSESSMENT': 13.894045,
 'CLOUD_SHADOW_PERCENTAGE': 0.268569,
 'DARK_FEATURES_PERCENTAGE': 2.128908,
 'DATASTRIP_ID': 'S2A_OPER_MSI_L2A_DS_SGS__20190926T202735_S20190926T160854_N02.13',
 'DATATAKE_IDENTIFIER': 'GS2A_20190926T160021_022259_N02.13',
 'DATATAKE_TYPE': 'INS-NOBS',
 'DEGRADED_MSI_DATA_PERCENTAGE': 0,
 'FORMAT_CORRECTNESS': 'PASSED',
 'GENERAL_QUALITY': 'PASSED',
 'GENERATION_TIME': 1569529655000,
 'GEOMETRIC_QUALITY': 'PASSED',
 'GRANULE_ID': 'L2A_T17RMQ_A022259_20190926T160854',
 'HIGH_PROBA_CLOUDS_PERCENTAGE': 0.036609,
 'IMAGE_DATE': '2019-09-26',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1': 104.552045558,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B10': 106.328758475,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B11': 105.114862299,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B12': 104.463844802,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2': 109.23026027,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3': 107.27178226,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4': 10

In [6]:
##Mapping
##Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(s2ga19, vis_param, "Sentinel 2")
#Map.addLayer(ga18, l8_param, "Landsat8SR")

Map.centerObject(ga_pts, 10)
#Map.set_plot_options(add_marker_cluster = True) ##not sure what this does


##add feature collection points for each plot
Map.addLayer(ga_pts)

Map

Map(center=[31.434120927658846, -81.34324597505625], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
##Python cloud-masking process for Sentinel-2 imagery: \
##https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless